In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import *
import numpy as np

In [2]:
#connector 설정

conf = SparkConf() \
    .setMaster('local') \
    .setAppName('sql') \
    .set("spark.driver.extraClassPath","jdbc/mysql-connector-java-5.1.44.jar")

sc = SparkContext(conf=conf)

spark = SparkSession(sc)

21/09/09 10:12:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/09/09 10:12:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#db접속정보 설정
                        #[ip주소:port]   #[db명]                 #[유저명]          #[비밀번호]
DB_URL = 'jdbc:mysql://                /       ?useSSL=false&user=        &password=           '


In [8]:
#데이터파일(csv)을 DataFrame 으로 읽어들인다.
# inferSchema : 데이터를 기반으로 타입을 유추(기본값 : False)
# header : 첫번째 행을 컬럼 값으로 사용

df = spark.read.csv("data/2017_2020_녹지율.csv", encoding = 'cp949',inferSchema = True, header = True)
df.show(10)

+-------------------+-------------------+----+--------+--------+--------+--------+----------+
|소재지(시군구)별(1)|소재지(시군구)별(2)|시점|주거지역|상업지역|공업지역|녹지지역|미지정지역|
+-------------------+-------------------+----+--------+--------+--------+--------+----------+
|소재지(시군구)별(1)|소재지(시군구)별(2)|시점|비율 (%)|비율 (%)|비율 (%)|비율 (%)|  비율 (%)|
|               전국|               소계|2017|   15.14|    1.88|    6.70|   71.54|      4.75|
|               전국|               소계|2018|   15.09|    1.88|    6.74|   70.99|      5.31|
|               전국|               소계|2019|   15.20|    1.90|    6.86|   71.11|      4.92|
|               전국|               소계|2020|   15.27|    1.90|    6.88|   71.04|      4.91|
|         서울특별시|               소계|2017|   53.82|    4.18|    3.30|   38.71|      0.00|
|         서울특별시|               소계|2018|   53.69|    4.23|    3.30|   38.78|      0.00|
|         서울특별시|               소계|2019|   53.68|    4.24|    3.30|   38.78|      0.00|
|         서울특별시|               소계|2020|   53.78|    4.24|    3.28|

In [5]:
#컬럼정보 확인
df.printSchema

<bound method DataFrame.printSchema of DataFrame[소재지(시군구)별(1): string, 소재지(시군구)별(2): string, 시점: string, 주거지역: string, 상업지역: string, 공업지역: string, 녹지지역: string, 미지정지역: string]>

In [6]:
# DataFrame 타입인 것을 확인
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [9]:
#DB에 저장
df \
    .write \
    .format('jdbc') \
    .mode('append') \
    .option("url", DB_URL) \
    .option("dbtable", "use_area2017_2020") \
    .save()